In [5]:
from sqlalchemy import create_engine
import pandas as pd
import re

In [6]:
engine = create_engine('sqlite:///testDB.sqlite3')
connection = engine.raw_connection()

построение таблицы для план-факт анализа

In [7]:
sql_pf_analysis = """WITH fact_table AS (
SELECT Bills.scID,
       strftime('%m', Bills.PayDate) as fact_month,
       strftime('%Y', Bills.PayDate) as fact_year,
       SUM(BC.Cost) as fact_sales
FROM Bills
JOIN BillsContent BC on Bills.bID = BC.bID
JOIN PriceItems PI on BC.piID = PI.piID
JOIN Agents A on Bills.scID = A.scID
WHERE PayDate IS NOT NULL AND product = 'Бухгалтерия.Контур' AND A.regionCode = 77
GROUP BY 1, 2, 3
),
plan_table AS (
SELECT rPlan.scID,
    rPlan.plan_amount,
    strftime('%m', rPlan.dt) as plan_month,
    strftime('%Y', rPlan.dt) as plan_year
FROM rPlan
LEFT JOIN Agents A on rPlan.scID = A.scID
WHERE product = 'Бухгалтерия.Контур' AND A.regionCode = 77
ORDER BY plan_year, plan_month
)
SELECT plan_table.scID,
       plan_amount as plan_sales,
       fact_sales,
       round(fact_sales - plan_amount) as abs_dev,
       round((fact_sales - plan_amount) / plan_amount * 100) as rel_dev,
       plan_year as year,
       plan_month as month
FROM plan_table
JOIN fact_table on plan_table.scID = fact_table.scID
               AND plan_table.plan_month = fact_table.fact_month
               AND plan_table.plan_year = fact_table.fact_year
"""
pf_analysis = pd.read_sql_query(sql_pf_analysis, connection)

print(pf_analysis)

                                     scID  plan_sales  fact_sales   abs_dev  \
0    1BCFA123-B3CC-FACF-E127-85FDBF3D386E    508300.0    488750.0  -19550.0   
1    3FED3285-975A-2D54-A448-3631D3F2D0A0    631078.3    624830.0   -6248.0   
2    49A9B9DC-EA97-4450-DFEB-473EB34753C2    396560.0    396560.0       0.0   
3    4C56CBA9-D127-1724-29A9-485F543E03B8    527817.6    549810.0   21992.0   
4    5F6C1C24-C72E-C3BF-F6E9-3A66B22C0530    516261.2    567320.0   51059.0   
..                                    ...         ...         ...       ...   
468  96BA5788-930E-CEB4-1868-6C4C8ED884DE   1435018.1   1420810.0  -14208.0   
469  AC52A184-7480-C0EF-8CF1-8AB6CA0C266C   1296486.0   1600600.0  304114.0   
470  C4CDA77A-511F-DE31-A3D1-EF8730E00339   1486408.6   1532380.0   45971.0   
471  C64FBAFC-1860-BAF2-D875-403150AF29AD   1414140.0   1414140.0       0.0   
472  D0A5BE04-FAAC-A4CE-1F17-7C3448093125   1855719.8   1974170.0  118450.0   

     rel_dev  year month  
0       -4.0  2015    06

в таблице PriceItems выделение в отдельные столбцы наименования и длительности тарифов для анализа с помощью регулярных выражений, далее присоединение данных по фактическим продажам к этой таблице

In [8]:
sql_tariff_naming = """SELECT piID, piName FROM PriceItems WHERE product = 'Бухгалтерия.Контур'"""
tariff_naming = pd.read_sql_query(sql_tariff_naming, connection)
print(tariff_naming)

    piID                                             piName
0     15  Доступ к сервису "Бухгалтерия.Контур" сроком н...
1    136  Доступ к сервису "Бухгалтерия.Контур" сроком н...
2    180  Доступ к сервису "Бухгалтерия.Контур" сроком н...
3    195  Доступ к сервису "Бухгалтерия.Контур" сроком н...
4    225  Доступ к сервису "Бухгалтерия.Контур" сроком н...
5    240  Доступ к сервису "Бухгалтерия.Контур" сроком н...
6    270  Доступ к сервису "Бухгалтерия.Контур" сроком н...
7     45  Доступ к сервису "Бухгалтерия.Контур" сроком н...
8     75  Доступ к сервису "Бухгалтерия.Контур" сроком н...
9    105  Доступ к сервису "Бухгалтерия.Контур" сроком н...
10   120  Доступ к сервису "Бухгалтерия.Контур" сроком н...


In [9]:
type_column = []
for name in tariff_naming.piName:
    a = re.search('[\"|(]([а-яА-Я]*)[\"|)]$', name).group(1)
    type_column.append(a)
tariff_naming['type'] = type_column
print(tariff_naming)

    piID                                             piName          type
0     15  Доступ к сервису "Бухгалтерия.Контур" сроком н...          Лайт
1    136  Доступ к сервису "Бухгалтерия.Контур" сроком н...       Базовый
2    180  Доступ к сервису "Бухгалтерия.Контур" сроком н...            ТД
3    195  Доступ к сервису "Бухгалтерия.Контур" сроком н...          Лайт
4    225  Доступ к сервису "Бухгалтерия.Контур" сроком н...            ТД
5    240  Доступ к сервису "Бухгалтерия.Контур" сроком н...          Лайт
6    270  Доступ к сервису "Бухгалтерия.Контур" сроком н...            ТД
7     45  Доступ к сервису "Бухгалтерия.Контур" сроком н...       Базовый
8     75  Доступ к сервису "Бухгалтерия.Контур" сроком н...  Максимальный
9    105  Доступ к сервису "Бухгалтерия.Контур" сроком н...            ТД
10   120  Доступ к сервису "Бухгалтерия.Контур" сроком н...          Лайт


In [10]:
duration_column = []
for name in tariff_naming.piName:
    a = re.search('сроком на\s*(\d* [\S]*)', name).group(1).strip()
    duration_column.append(a)
tariff_naming['duration'] = duration_column
print(tariff_naming)

    piID                                             piName          type  \
0     15  Доступ к сервису "Бухгалтерия.Контур" сроком н...          Лайт   
1    136  Доступ к сервису "Бухгалтерия.Контур" сроком н...       Базовый   
2    180  Доступ к сервису "Бухгалтерия.Контур" сроком н...            ТД   
3    195  Доступ к сервису "Бухгалтерия.Контур" сроком н...          Лайт   
4    225  Доступ к сервису "Бухгалтерия.Контур" сроком н...            ТД   
5    240  Доступ к сервису "Бухгалтерия.Контур" сроком н...          Лайт   
6    270  Доступ к сервису "Бухгалтерия.Контур" сроком н...            ТД   
7     45  Доступ к сервису "Бухгалтерия.Контур" сроком н...       Базовый   
8     75  Доступ к сервису "Бухгалтерия.Контур" сроком н...  Максимальный   
9    105  Доступ к сервису "Бухгалтерия.Контур" сроком н...            ТД   
10   120  Доступ к сервису "Бухгалтерия.Контур" сроком н...          Лайт   

   duration  
0    6 мес.  
1     1 год  
2    2 мес.  
3   18 мес.  
4    

In [11]:
sql_for_tariff_analysis = """SELECT C.cID,
       cName,
       piName,
       strftime('%m', Bills.PayDate) as fact_month,
       strftime('%Y', Bills.PayDate) as fact_year,
       SUM(BC.Cost)
FROM Bills
         JOIN BillsContent BC on Bills.bID = BC.bID
         JOIN PriceItems PI on BC.piID = PI.piID
         JOIN Agents A on Bills.scID = A.scID
         join Clients C on Bills.cID = C.cID
WHERE PayDate IS NOT NULL AND product = 'Бухгалтерия.Контур' AND A.regionCode = 77
GROUP BY 1, 2, 3, 4, 5"""
tariff_analysis = pd.read_sql_query(sql_for_tariff_analysis, connection)
print(tariff_analysis)

          cID               cName  \
0          20    Michael Catalano   
1          36         Carly Zhang   
2          36         Carly Zhang   
3          36         Carly Zhang   
4          38           Helen Lal   
...       ...                 ...   
46075  842009        Annik Rousey   
46076  842009        Annik Rousey   
46077  842011    Connie O'Connell   
46078  842013  Christina Chisholm   
46079  842041     Stefen Bischoff   

                                                  piName fact_month fact_year  \
0      Доступ к сервису "Бухгалтерия.Контур" сроком н...         07      2018   
1      Доступ к сервису "Бухгалтерия.Контур" сроком н...         07      2016   
2      Доступ к сервису "Бухгалтерия.Контур" сроком н...         06      2016   
3      Доступ к сервису "Бухгалтерия.Контур" сроком н...         10      2016   
4      Доступ к сервису "Бухгалтерия.Контур" сроком н...         03      2016   
...                                                  ...        ...  

In [12]:
merge_tariff = pd.merge(tariff_analysis, tariff_naming, on = 'piName')
merge_tariff

,cID,cName,piName,fact_month,fact_year,SUM(BC.Cost),piID,type,duration
0,20,Michael Catalano,"Доступ к сервису ""Бухгалтерия.Контур"" сроком н...",07,2018,19500.0,240,Лайт,2 года
1,36,Carly Zhang,"Доступ к сервису ""Бухгалтерия.Контур"" сроком н...",06,2016,16380.0,240,Лайт,2 года
2,38,Helen Lal,"Доступ к сервису ""Бухгалтерия.Контур"" сроком н...",11,2016,15990.0,240,Лайт,2 года
3,72,Gregory Spicer,"Доступ к сервису ""Бухгалтерия.Контур"" сроком н...",04,2016,16380.0,240,Лайт,2 года
4,72,Gregory Spicer,"Доступ к сервису ""Бухгалтерия.Контур"" сроком н...",09,2016,35100.0,240,Лайт,2 года
...,...,...,...,...,...,...,...,...,...
46075,805130,Reinout Blanton,"Доступ к сервису ""Бухгалтерия.Контур"" сроком н...",06,2016,0.0,270,ТД,4 мес.
46076,823169,Victor Canuto,"Доступ к сервису ""Бухгалтерия.Контур"" сроком н...",11,2017,0.0,270,ТД,4 мес.
46077,824945,Devin Cao,"Доступ к сервису ""Бухгалтерия.Контур"" сроком н...",10,2016,0.0,270,ТД,4 мес.
46078,832398,Priscilla Waggoner,"Доступ к сервису ""Бухгалтерия.Контур"" сроком н...",12,2015,0.0,270,ТД,4 мес.
